In [ ]:
import pandas as pd

df = pd.read_excel('C://Users//Halil//Pythons//CourseProgramsGenerator//data//input//course_offered_2425F.xls')

In [ ]:
courses = {}
for index, row in df.iterrows():
    subject = str(row['SUBJECT']).strip() if pd.notna(row['SUBJECT']) else None
    course_no = str(row['COURSENO']).strip() if pd.notna(row['COURSENO']) else None
    section_no = str(row['SECTIONNO']).strip() if pd.notna(row['SECTIONNO']) else None
    title = str(row['TITLE']).strip() if pd.notna(row['TITLE']) else None
    faculty = str(row['FACULTY']).strip() if pd.notna(row['FACULTY']) else None
    ects_credits = row['CREDITS'] if pd.notna(row['CREDITS']) else None
    instructor_full_name = str(row['INSTRUCTORFULLNAME']).strip() if pd.notna(row['INSTRUCTORFULLNAME']) else None
    corequisites = str(row['COREQUISITE']).strip() if pd.notna(row['COREQUISITE']) else None
    prequisites = str(row['PREREQUISITE']).strip() if pd.notna(row['PREREQUISITE']) else None
    description = str(row['DESCRIPTION']).strip() if pd.notna(row['DESCRIPTION']) else None
    schedule_for_print = str(row['SCHEDULEFORPRINT']).strip() if pd.notna(row['SCHEDULEFORPRINT']) else None

    corequisite_list = [corequisite for corequisite in corequisites.split(" and ")] if corequisites else []

    schedule_list = []
    if schedule_for_print:
        for time_slots in schedule_for_print.split("\n"):
            print(schedule_for_print)
            day, interval = time_slots.split(" | ")
            interval = interval.replace(":", ".")
            start_time, end_time = interval.split(" - ")
            schedule_list.append({
                "day": day,
                "interval": interval}
            )

    course_code = f"{subject} {course_no}.{section_no}"
    course_id = f"{subject} {course_no}"

    courses[course_code] = {

        "course_code": course_code,
        "course_name": title,
        "credits": ects_credits,
        "schedule": schedule_list,
        "subject": subject,
        "course_no": course_no,
        "section_no": section_no,
        "course_id": course_id,
        "faculty": faculty,
        "instructor_full_name": instructor_full_name,
        "corequisites": corequisite_list,
        "prequisites": prequisites,
        "description": description,
    }
print(courses)